# Cleaner
The goal of this notebook is to combine and clean the data.

In [3]:
import pandas as pd
import numpy as np

The first function just reads the data off the csv files.

In [4]:
def read_file(year:int):#reads data from a csv file and puts it in dataframe
    df0=pd.read_csv(f"Houses build in {year}")
    return(df0)


First we read all the csv files produced by the scraper and we combine them in a dataframe. We then drop any extra columns, reset the indexing and drop any entries with n\a values.

In [5]:
df0=read_file(1952)
for i in range(1953,2023):
    df1=read_file(i)
    df0=pd.concat([df0,df1],ignore_index=True)


df0=df0.drop(columns=["Unnamed: 0"])

df0=df0.dropna()
df0=df0.reset_index(drop=True)

We find the set of all possible locations for the houses on sale.

In [6]:
df0["Location"].unique()

array(['Λευκός Πύργος (Κέντρο Θεσσαλονίκης)',
       'Βαρδάρης (Βαρδάρης - Λαχανόκηποι)',
       'Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης)',
       'Αγία Σοφία (Κέντρο Θεσσαλονίκης)',
       'Αντιγονιδών (Κέντρο Θεσσαλονίκης)',
       'Καμάρα - Ροτόντα (Κέντρο Θεσσαλονίκης)',
       'Διοικητήριο (Κέντρο Θεσσαλονίκης)',
       'Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης)',
       'Βούλγαρη - Άγιος Ελευθέριος - Ντεπώ (Βούλγαρη - Ντεπώ - Μαρτίου)',
       'Κέντρο Θεσσαλονίκης (Θεσσαλονίκη - Δήμος)',
       'Ξηροκρήνη (Ξηροκρήνη - Παναγία Φανερωμένη)', 'Κέντρο (Πανόραμα)',
       'ΔΕΘ - Πανεπιστήμια (Κέντρο Θεσσαλονίκης)',
       'Φάληρο - Ιπποκράτειο (Θεσσαλονίκη - Δήμος)',
       'Ασβεστοχώρι (Χορτιάτης)', 'Τσινάρι (Άνω Πόλη)',
       'Επτάλοφος (Αμπελόκηποι)', 'Κέντρο (Συκιές)', 'Κέντρο (Κουφάλια)',
       'Κέντρο (Μενεμένη)', 'Αγία Τριάδα (Φάληρο - Ιπποκράτειο)',
       'Ζαγκλιβέρι (Καλλίνδοια)', 'Κέντρο (Πεύκα)',
       'Άνω Πόλη (Θεσσαλονίκη - Δήμος)', 'Άγιοι Θεόδωροι (Συκιές)',
       'Λητή 

In [7]:
df0.head()

,Location,Price,Total_area,House_type,Floor,Rooms,Bathrooms,Year_of_construction
0,Λευκός Πύργος (Κέντρο Θεσσαλονίκης),€2.700.000,170τ.μ.,Μεζονέτα,7,3.0,2.0,1952.0
1,Βαρδάρης (Βαρδάρης - Λαχανόκηποι),€245.000,70τ.μ.,Διαμέρισμα,7,2.0,1.0,1952.0
2,Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης),€245.000,70τ.μ.,Διαμέρισμα,7,2.0,1.0,1952.0
3,Βαρδάρης (Βαρδάρης - Λαχανόκηποι),€245.000,70τ.μ.,Διαμέρισμα,7,2.0,1.0,1952.0
4,Αγία Σοφία (Κέντρο Θεσσαλονίκης),€290.000,71τ.μ.,Διαμέρισμα,1,2.0,1.0,1952.0


We notice that there are too many possible locations with significant overlap. We need to tidy things up. We will create 27 custom locations based on the above list. The choice was made based on the areas visible on google maps. The dict_loc dictionary will contain each of these locations as keys. The values of the dictionary will be a list which contains their corresponding instances in the above list.

In [8]:
dict_loc={"Center_east_of_aristotle_square" : [
    "Λευκός Πύργος (Κέντρο Θεσσαλονίκης)",
    "ΔΕΘ - Πανεπιστήμια (Κέντρο Θεσσαλονίκης)",
   "Καμάρα - Ροτόντα (Κέντρο Θεσσαλονίκης)",
    "Αγία Σοφία (Κέντρο Θεσσαλονίκης)",'ΔΕΘ - Πανεπιστήμια (Κέντρο Θεσσαλονίκης)',
]

,"Vardaris_xirokrini" :[
    "Ξηροκρήνη - Παναγία Φανερωμένη (Θεσσαλονίκη - Δήμος)",'Λαχανόκηποι (Βαρδάρης - Λαχανόκηποι)','Ξηροκρήνη (Ξηροκρήνη - Παναγία Φανερωμένη)', 
    "Σφαγεία - Ιχθυόσκαλα (Θεσσαλονίκη - Δήμος)",'Παναγία Φανερωμένη (Ξηροκρήνη - Παναγία Φανερωμένη)',
    'Βαρδάρης (Βαρδάρης - Λαχανόκηποι)','Βαρδάρης - Λαχανόκηποι (Θεσσαλονίκη - Δήμος)',
    'Παλαιός Σιδηροδρομικός Σταθμός (Σφαγεία - Ιχθυόσκαλα)','Ξηροκρήνη - Παναγία Φανερωμένη (Θεσσαλονίκη - Δήμος)','Σταθμός ΟΣΕ (Βαρδάρης - Λαχανόκηποι)', 
],

"Center_west_of_including_aristotle_square" : [ 'Λαδάδικα (Κέντρο Θεσσαλονίκης)','Κέντρο Θεσσαλονίκης (Θεσσαλονίκη - Δήμος)',
    "Αντιγονιδών (Κέντρο Θεσσαλονίκης)",
     "Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης)",
    'Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης)',
    "Διοικητήριο (Κέντρο Θεσσαλονίκης)",
    "Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης)",
    "Κέντρο Θεσσαλονίκης (Θεσσαλονίκη - Δήμος)",
     'Διοικητήριο (Κέντρο Θεσσαλονίκης)',
    "Λαδάδικα (Κέντρο Θεσσαλονίκης)"
],

"Voulgari_Ntepo" : [
    'Βούλγαρη - Άγιος Ελευθέριος - Ντεπώ (Βούλγαρη - Ντεπώ - Μαρτίου)',
       "Βούλγαρη - Ντεπώ - Μαρτίου (Θεσσαλονίκη - Δήμος)",'Βούλγαρη - Ντεπώ - Μαρτίου (Θεσσαλονίκη - Δήμος)','Μαρτίου (Βούλγαρη - Ντεπώ - Μαρτίου)', 
],

"Panorama" : [ 'Κέντρο (Πανόραμα)','Συνοικισμός Νόμου 751 (Πανόραμα)','Παλιός Οικισμός Πανοράματος (Πανόραμα)', 'Οικισμός Μακεδονία (Πανόραμα)','Οικισμός Τοπογράφων (Πανόραμα)', 'Πανόραμα (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
    
],

"Agia_triada" : ['Φάληρο - Ιπποκράτειο (Θεσσαλονίκη - Δήμος)','Αγία Τριάδα (Φάληρο - Ιπποκράτειο)','Ιπποκράτειο (Φάληρο - Ιπποκράτειο)', ],

"Ano_poli_Evaggelistria" : ['Τσινάρι (Άνω Πόλη)','Κάστρα (Άνω Πόλη)', 'Άνω Πόλη (Θεσσαλονίκη - Δήμος)', 
    '40 Εκκλησιές (40 Εκκλησιές - Ευαγγελίστρια)',"Άνω Πόλη (Θεσσαλονίκη - Δήμος)", 'Άγιος Παύλος (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Πανεπιστημιούπολη (Άγιος Παύλος)',
    "40 Εκκλησιές - Ευαγγελίστρια (Θεσσαλονίκη - Δήμος)",'Χίλια Δέντρα (Άγιος Παύλος)', 'Γεντί Κουλέ (Άγιος Παύλος)','Ευαγγελίστρια (40 Εκκλησιές - Ευαγγελίστρια)',
],

"Ampelokipoi" : ['Επτάλοφος (Αμπελόκηποι)', 'Σκεπάρνη (Αμπελόκηποι)', 'Μεταγωγών (Αμπελόκηποι)',
     'Αμπελόκηποι (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
       'Βόσπορος (Αμπελόκηποι)','Δημαρχείου (Αμπελόκηποι)', 'Φιλλίπου (Αμπελόκηποι)',"Αμπελόκηποι (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Ζωοδόχου Πηγής (Αμπελόκηποι)',
       
],

"Sykies" : [ 'Κέντρο (Συκιές)', 'Άγιοι Θεόδωροι (Συκιές)','Άλσος (Συκιές)', 'Δροσιά (Συκιές)', 'Συκιές (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
            'Μαύρος Γάτος (Συκιές)', 'Παράσχου (Συκιές)', 'Καλλιθέα (Συκιές)','Επταπύργιο (Συκιές)', 'Ροδοχώρι (Συκιές)',],

"Menemeni_lachanokipoi" : ['Κέντρο (Μενεμένη)', 
    "Μενεμένη (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Λαχανόκηποι (Μενεμένη)','Μενεμένη (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
],

"Peuka": ['Κέντρο (Πεύκα)','Πεύκα (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Στροφή Φιλύρου (Πεύκα)','Δελασάλ (Πεύκα)',],

"Evosmos_Kordelio" : ['Κέντρο (Ελευθέριο-Κορδελιό)','Ευαγγελισμός - Νέος Κουκλουτζάς (Εύοσμος)','Εύοσμο (Εύοσμος)',
    'Κέντρο (Εύοσμος)', 'Κάτω Ηλιούπολη (Εύοσμος)', 'Νέο Κορδελιό (Ελευθέριο-Κορδελιό)',"Εύοσμος (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Νέα Πολιτεία (Εύοσμος)','Άνωθεν Περιφερειακού (Εύοσμος)', 'Ελευθέριο-Κορδελιό (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
       
],

"Stavroupoli" : ['Ομόνοια (Σταυρούπολη)','Άνωθεν Ασύλου (Σταυρούπολη)',
    'Πρόνοια (Σταυρούπολη)','Σταυρούπολη (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Ανθόκηποι (Σταυρούπολη)',"Σταυρούπολη (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Ακριτών (Σταυρούπολη)', 'Κέντρο (Σταυρούπολη)',  'Αμπελώνες (Σταυρούπολη)', 
       'Τερψιθέα (Σταυρούπολη)','Κάτω Ηλιούπολη (Σταυρούπολη)','Άνω Ηλιούπολη (Σταυρούπολη)',
],

"Neapoli" : ['Νεάπολη (Νεάπολη)','Άνω Αναγέννηση (Νεάπολη)','Πυροπαθών (Νεάπολη)','Ρήγα Φεραίου (Νεάπολη)','Κέντρο (Νεάπολη)','Κάτω Αναγέννηση (Νεάπολη)','Άγιος Γιώργιος - Τρώαδα (Νεάπολη)','Νεάπολη (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Στρεμπενιώτη (Νεάπολη)','Κόκκορας (Νεάπολη)','Κουντουριώτη (Νεάπολη)', 
         ],

"Analipsi" : [     'Ανάληψη (Ανάληψη - Μπότσαρη - Νέα Παραλία)', 
       'Μπότσαρη (Ανάληψη - Μπότσαρη - Νέα Παραλία)', 'Ανάληψη - Μπότσαρη - Νέα Παραλία (Θεσσαλονίκη - Δήμος)','Νέα Παραλία (Ανάληψη - Μπότσαρη - Νέα Παραλία)',],

"Toumpa" : ['Άνω Τούμπα (Τούμπα)','Παπάφη (Τούμπα)',
       'Τούμπα (Θεσσαλονίκη - Δήμος)', 'Μαλακοπή (Τούμπα)','Κάτω Τούμπα (Τούμπα)',],

"Xarilaou" : [ 'Χαριλάου (Θεσσαλονίκη - Δήμος)','Οσία Ξένη (Χαριλάου)',  'Βρυσάκι (Χαριλάου)', 'Αγία Κυριακή (Χαριλάου)','Άγιος Νικόλαος (Χαριλάου)', 'Νέα Ελβετία (Χαριλάου)', ],

"Kalamaria" : [ 'Καραμπουρνακι (Καλαμαριά)','Φοίνικας (Καλαμαριά)', 'Άγιος Ιωάννης (Καλαμαριά)','Νέα Κρήνη (Καλαμαριά)', 'Αρετσού (Καλαμαριά)','Κηφισιά (Καλαμαριά)', 
    'Κουρί (Καλαμαριά)',
       'Κέντρο (Καλαμαριά)', 'Βότση (Καλαμαριά)', 'Βυζάντιο (Καλαμαριά)', 'Καλαμαριά (Θεσσαλονίκη - Περιφ/κοί δήμοι)',"Καλαμαριά (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Επέκταση (Καλαμαριά)', 'Δέρκων (Καλαμαριά)','Άγιος Παντελεήμων (Καλαμαριά)', 
],

"Pylaia" : [
    'Πυλαία (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Κέντρο (Πυλαία)','Επέκταση (Πυλαία)','Ελαιώνες (Πυλαία)', 'Βίλλα Ριτζ (Πυλαία)','Πυλαιώτικα (Πυλαία)',"Πυλαία (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Τοπογράφοι (Πυλαία)','Mediterranean Cosmos (Πυλαία)', 'Πουρνάρι (Πυλαία)','Κωνσταντινοπολίτικα (Πυλαία)','Άνω Μαλακοπή (Πυλαία)',
],

"Peraia" : ['Περαία (Θερμαϊκός)',],

"Triandria" : [
    'Δόξα (Τριανδρία - Δόξα)',"Τριανδρία - Δόξα (Θεσσαλονίκη - Δήμος)",'Τριανδρία - Δόξα (Θεσσαλονίκη - Δήμος)', 'Τριανδρία (Τριανδρία - Δόξα)',
],

"Polichni" : [ 'Επέκταση (Πολίχνη)','Κέντρο (Πολίχνη)', 'Μετέωρα (Πολίχνη)','Άγιος Ραφαήλ (Πολίχνη)', 'Καρατάσσου (Πολίχνη)', 'Άγιος Ιωάννης (Πολίχνη)',
            'Άνω Πολίχνη (Πολίχνη)','Άγιος Παντελεήμονας (Πολίχνη)','Πολίχνη (Θεσσαλονίκη - Περιφ/κοί δήμοι)', ],

"Efkarpia_Nikopoli" : [
    'Νικόπολη (Σταυρούπολη)','Κέντρο (Ευκαρπία)',"Ευκαρπία (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Ευξεινούπολη Ευκαρπίας (Ευκαρπία)',
],

"Oreokastro" : [
    'Κέντρο (Ωραιόκαστρο)','Γαλήνη (Ωραιόκαστρο)', "Ωραιόκαστρο (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'ΒΙ.ΠΑ. (Ωραιόκαστρο)','Παλαιόκαστρο (Ωραιόκαστρο)', 
    'Ασπρόβρυση (Ωραιόκαστρο)', 'Αμφιθέα (Ωραιόκαστρο)','Ωραιόκαστρο (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
],

"Thermi" : [
    "Θέρμη (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Περιοχή Αεροδρομίου (Θέρμη)','Ταγαράδες (Θέρμη)','Νέα Ραιδεστός (Θέρμη)','Κέντρο (Θέρμη)', 'Νέο Ρύσιο (Θέρμη)', 'Τριάδιο (Θέρμη)',
],

"Sindos_Diavata" : [
    "Σίνδος (Εχέδωρος)",
    "Νέα Μαγνησία (Εχέδωρος)",
    "Καλοχώρι (Εχέδωρος)",
    "Αξιός (Θεσσαλονίκη - Υπόλ. Νομού)",'Διαβατά (Εχέδωρος)',
    
],

"out_of_thessaloniki" : [
    "Κουφάλια (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Χαλκηδόνα (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Άγιος Αθανάσιος (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Κορώνεια (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Λαγκαδάς (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Χορτιάτης (Θεσσαλονίκη - Περιφ/κοί δήμοι)"
]}


Now we create a list which will contain the aproximate location of each entry on our dataframe. We append this list to our dataframe.

In [9]:
loc_aprox=[]
for j in range(len(df0["Location"])):
    other=True
    for i in dict_loc:
        if df0["Location"][j] in dict_loc[i]:
            loc_aprox.append(i)
            other=False
    if other==True:
        loc_aprox.append("out_of_thessaloniki")


In [10]:
df0["Location_aprox"]=loc_aprox

We also notice that the floor column has non-numeric values. We need to fix that so we will create a new column with numeric values corresponding to the categorical ones. The correspondance goes as follows ΗΜ means mezzanine so it corresponds to 0.5, ΗΜ/ΥΠ means semi-basement so it corresponds to -0.5, ΙΣ means ground floor so it corresponds to 0 and ΥΠ means basement so it corresponds to -1.

In [11]:
df0["Floor"].unique()

array(['7', '1', '6', '3', '4', 'ΗΜ', 'ΗΜ/ΥΠ', '2', 'ΙΣ', '5', 'ΥΠ', '8',
       '9', '10', '12'], dtype=object)

We remove any non numeric characters from our columns.

In [12]:

def clean_data(df0):

    df0.insert(2, "Price_cleaned", df0["Price"].str.replace(".", ""))
    df0.insert(2, "Price2", df0["Price_cleaned"].str.replace("€", ""))
    df0.insert(2, "Total_area2", df0["Total_area"].str.replace("τ.μ.", ""))
    
    return df0

df1 = clean_data(df0.copy())
df1.head()
df1=df1.drop(columns=["Price","Price_cleaned","Total_area"])


In [13]:
df1=df1.rename(columns={"Price2":"Price","Total_area2":"Total_area"})
df1=df1.drop(columns=["Location"])


In [14]:
df1["Floor"].unique()

array(['7', '1', '6', '3', '4', 'ΗΜ', 'ΗΜ/ΥΠ', '2', 'ΙΣ', '5', 'ΥΠ', '8',
       '9', '10', '12'], dtype=object)

In [15]:
df1.dtypes

Total_area               object
Price                    object
House_type               object
Floor                    object
Rooms                   float64
Bathrooms               float64
Year_of_construction    float64
Location_aprox           object
dtype: object

Finally we save the dataframe as a csv file.

In [17]:
df0.to_csv("Houses_augmented2")

In [29]:
df0.groupby(["Location_aprox"]).count()

,Location,Price,Total_area,House_type,Floor,Rooms,Bathrooms,Year_of_construction
Location_aprox,,,,,,,,
Agia_triada,3072,3072,3072,3072,3072,3072,3072,3072
Ampelokipoi,918,918,918,918,918,918,918,918
Analipsi,2645,2645,2645,2645,2645,2645,2645,2645
Ano_poli_Evaggelistria,1859,1859,1859,1859,1859,1859,1859,1859
Center_east_of_aristotle_square,5818,5818,5818,5818,5818,5818,5818,5818
Center_west_of_including_aristotle_square,5429,5429,5429,5429,5429,5429,5429,5429
Efkarpia_Nikopoli,467,467,467,467,467,467,467,467
Evosmos_Kordelio,2644,2644,2644,2644,2644,2644,2644,2644
Kalamaria,4589,4589,4589,4589,4589,4589,4589,4589


In [23]:
df1=df0.groupby(["Location_aprox"])

In [28]:
df0["Total_area"][1].replace("τ.μ."," ")

' 70 '

In [31]:
for j in range(len(df0["Total_area"])):
    df0.loc[j,"Total_area"]=df0["Total_area"][j].replace("τ.μ.", " ")

In [32]:
df0

,Location,Price,Total_area,House_type,Floor,Rooms,Bathrooms,Year_of_construction,Location_aprox
0,Λευκός Πύργος (Κέντρο Θεσσαλονίκης),€2.700.000,170,Μεζονέτα,7,3.0,2.0,1952.0,Center_east_of_aristotle_square
1,Βαρδάρης (Βαρδάρης - Λαχανόκηποι),€245.000,70,Διαμέρισμα,7,2.0,1.0,1952.0,Vardaris_xirokrini
2,Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης),€245.000,70,Διαμέρισμα,7,2.0,1.0,1952.0,Center_west_of_including_aristotle_square
3,Βαρδάρης (Βαρδάρης - Λαχανόκηποι),€245.000,70,Διαμέρισμα,7,2.0,1.0,1952.0,Vardaris_xirokrini
4,Αγία Σοφία (Κέντρο Θεσσαλονίκης),€290.000,71,Διαμέρισμα,1,2.0,1.0,1952.0,Center_east_of_aristotle_square
...,...,...,...,...,...,...,...,...,...
63026,Κάτω Τούμπα (Τούμπα),€485.000,138,Μεζονέτα,8,4.0,2.0,2022.0,Toumpa
63027,Άνω Τούμπα (Τούμπα),€370.000,156,Μεζονέτα,6,2.0,1.0,2022.0,Toumpa
63028,Εύοσμος (Θεσσαλονίκη - Περιφ/κοί δήμοι),€180.000,87,Διαμέρισμα,1,2.0,1.0,2022.0,Evosmos_Kordelio
63029,Τριανδρία (Τριανδρία - Δόξα),€370.000,153,Μεζονέτα,6,4.0,1.0,2022.0,Triandria


In [6]:
df3=pd.read_csv("All houses augmented")

In [34]:

def clean_data(df3):
    # Drop column: 'Unnamed: 0'
    df3 = df3.drop(columns=['Unnamed: 0'])
    # Drop column: 'index'
    df3 = df3.drop(columns=['index'])
    df3=df3.drop(columns=["Location"])
    df3=df3.replace({"Λοιπές κατηγορίες":"Διαμέρισμα"},regex=True)
    return df3

df3_clean = clean_data(df3.copy())
df3_clean.head()

,Price,Total_area,House_type,Floor,Rooms,Bathrooms,Year_of_construction,Location_aprox
0,2700000.0,170.0,Μεζονέτα,7.0,3.0,2.0,1952.0,Center east of aristotle square
1,245000.0,70.0,Διαμέρισμα,7.0,2.0,1.0,1952.0,Vardaris-lachanokipi-xirokrini
2,145000.0,42.0,Διαμέρισμα,6.0,1.0,1.0,1952.0,Center west of (including) aristotle square
3,245000.0,70.0,Διαμέρισμα,7.0,2.0,1.0,1952.0,Vardaris-lachanokipi-xirokrini
4,245000.0,70.0,Διαμέρισμα,7.0,2.0,1.0,1952.0,Vardaris-lachanokipi-xirokrini


In [ ]:
df3_clean

In [22]:
df3.groupby("House_type").count()

,Unnamed: 0,index,Location,Price,Total_area,Floor,Rooms,Bathrooms,Year_of_construction,Location_aprox
House_type,,,,,,,,,,
Loft,201,201,201,201,201,201,201,201,201,201
Studio / Γκαρσονιέρα,8924,8924,8924,8924,8924,8924,8924,8924,8924,8924
Βίλα,575,575,575,575,575,575,575,575,575,575
Διαμέρισμα,38586,38586,38586,38586,38586,38586,38586,38586,38586,38586
Κτίριο,237,237,237,237,237,237,237,237,237,237
Λοιπές κατηγορίες,8,8,8,8,8,8,8,8,8,8
Μεζονέτα,6748,6748,6748,6748,6748,6748,6748,6748,6748,6748
Μονοκατοικία,3862,3862,3862,3862,3862,3862,3862,3862,3862,3862
Συγκρότημα διαμερισμάτων,164,164,164,164,164,164,164,164,164,164


In [24]:
df4=

In [25]:
df4["House_type"].unique()

array(['Μεζονέτα', 'Διαμέρισμα', 'Studio / Γκαρσονιέρα', 'Μονοκατοικία',
       'Συγκρότημα διαμερισμάτων', 'Κτίριο', 'Loft', 'Βίλα'], dtype=object)

In [31]:
df5=df3_clean.loc[df3_clean["Location"]=="Αγία Σοφία (Κέντρο Θεσσαλονίκης)"]

In [32]:
df5.groupby(df5.columns.tolist(),as_index=False).size()

,Location,Price,Total_area,House_type,Floor,Rooms,Bathrooms,Year_of_construction,Location_aprox,size
0,Αγία Σοφία (Κέντρο Θεσσαλονίκης),55000.0,25.0,Studio / Γκαρσονιέρα,1.0,1.0,1.0,1965.0,Center east of aristotle square,1
1,Αγία Σοφία (Κέντρο Θεσσαλονίκης),62000.0,41.0,Διαμέρισμα,-0.5,1.0,1.0,1957.0,Center east of aristotle square,2
2,Αγία Σοφία (Κέντρο Θεσσαλονίκης),79999.0,50.0,Διαμέρισμα,0.0,1.0,1.0,1960.0,Center east of aristotle square,8
3,Αγία Σοφία (Κέντρο Θεσσαλονίκης),80000.0,40.0,Studio / Γκαρσονιέρα,0.0,1.0,1.0,1961.0,Center east of aristotle square,8
4,Αγία Σοφία (Κέντρο Θεσσαλονίκης),82000.0,56.0,Studio / Γκαρσονιέρα,-0.5,1.0,1.0,1953.0,Center east of aristotle square,4
...,...,...,...,...,...,...,...,...,...,...
166,Αγία Σοφία (Κέντρο Θεσσαλονίκης),635000.0,130.0,Διαμέρισμα,2.0,3.0,2.0,1979.0,Center east of aristotle square,16
167,Αγία Σοφία (Κέντρο Θεσσαλονίκης),635000.0,140.0,Διαμέρισμα,3.0,3.0,1.0,1978.0,Center east of aristotle square,16
168,Αγία Σοφία (Κέντρο Θεσσαλονίκης),800000.0,197.0,Διαμέρισμα,1.0,8.0,3.0,1960.0,Center east of aristotle square,4
169,Αγία Σοφία (Κέντρο Θεσσαλονίκης),1000000.0,165.0,Διαμέρισμα,8.0,4.0,1.0,1976.0,Center east of aristotle square,1


In [30]:
df5

,Unnamed: 0,index,Location,Price,Total_area,House_type,Floor,Rooms,Bathrooms,Year_of_construction,Location_aprox
27,27,27,Αγία Σοφία (Κέντρο Θεσσαλονίκης),175000.0,37.0,Studio / Γκαρσονιέρα,6.0,1.0,1.0,1953.0,Center east of aristotle square
29,29,29,Αγία Σοφία (Κέντρο Θεσσαλονίκης),135000.0,40.0,Διαμέρισμα,4.0,1.0,1.0,1953.0,Center east of aristotle square
30,30,30,Αγία Σοφία (Κέντρο Θεσσαλονίκης),435000.0,75.0,Διαμέρισμα,6.0,2.0,1.0,1953.0,Center east of aristotle square
31,31,31,Αγία Σοφία (Κέντρο Θεσσαλονίκης),435000.0,75.0,Διαμέρισμα,6.0,2.0,1.0,1953.0,Center east of aristotle square
35,35,35,Αγία Σοφία (Κέντρο Θεσσαλονίκης),135000.0,35.0,Διαμέρισμα,4.0,1.0,1.0,1953.0,Center east of aristotle square
...,...,...,...,...,...,...,...,...,...,...,...
52275,52275,54581,Αγία Σοφία (Κέντρο Θεσσαλονίκης),190000.0,51.0,Διαμέρισμα,2.0,1.0,1.0,2006.0,Center east of aristotle square
52300,52300,54609,Αγία Σοφία (Κέντρο Θεσσαλονίκης),190000.0,51.0,Διαμέρισμα,2.0,1.0,1.0,2006.0,Center east of aristotle square
52352,52352,54667,Αγία Σοφία (Κέντρο Θεσσαλονίκης),190000.0,51.0,Διαμέρισμα,2.0,1.0,1.0,2006.0,Center east of aristotle square
52377,52377,54695,Αγία Σοφία (Κέντρο Θεσσαλονίκης),190000.0,51.0,Διαμέρισμα,2.0,1.0,1.0,2006.0,Center east of aristotle square
